In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset = dataset.drop('User ID' , axis = 1)

In [8]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [9]:
dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [10]:
indep = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

dep = dataset['Purchased']

In [11]:
indep.shape

(400, 3)

In [12]:
        indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [13]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators' : [10,100],
              'criterion': ['gini', 'entropy', 'log_loss']
              }
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit = 'True', verbose = 3,n_jobs = -1, scoring = 'f1')
#fiiting the model for grid
grid.fit(X_train,Y_train)



Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\joblib\parallel.py", line 921, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\Admin\Anaconda3\lib\site-p

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'n_estimators': [10, 100]},
             refit='True', scoring='f1', verbose=3)

In [17]:
#print best parameter after tuning 
#print(grid.best_params_)

re = grid.cv_results_

grid_predictions = grid.predict(X_test)



from sklearn.metrics import confusion_matrix
cm = confusion_matrix (Y_test,grid_predictions)

#print classification report

from sklearn.metrics import classification_report

clf_report  = classification_report(Y_test,grid_predictions)

In [18]:
from sklearn.metrics import f1_score

f1_macro = f1_score(Y_test,grid_predictions,average = 'weighted')

print('The R_score vlaue for best parameter {}:'.format(grid.best_params_),f1_score)

The R_score vlaue for best parameter {'criterion': 'entropy', 'n_estimators': 100}: <function f1_score at 0x00000129FB7838B8>


In [19]:
print (cm)

[[74  5]
 [ 5 36]]


In [20]:
print (clf_report)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        79
           1       0.88      0.88      0.88        41

    accuracy                           0.92       120
   macro avg       0.91      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [21]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,-1])

0.958783575177524

In [22]:
table = pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.470443,0.130004,0.060765,0.032902,gini,10,"{'criterion': 'gini', 'n_estimators': 10}",0.756757,0.800000,0.761905,0.883721,0.975610,0.835598,0.083466,2
1,1.456967,0.371672,0.157110,0.058343,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",0.820513,0.717949,0.790698,0.883721,0.952381,0.833052,0.080008,3
2,0.090946,0.035705,0.021988,0.007585,entropy,10,"{'criterion': 'entropy', 'n_estimators': 10}",0.789474,0.769231,0.790698,0.857143,0.810811,0.803471,0.029887,4
3,0.518704,0.036953,0.068760,0.014951,entropy,100,"{'criterion': 'entropy', 'n_estimators': 100}",0.850000,0.829268,0.790698,0.883721,0.926829,0.856103,0.046476,1
4,0.023987,0.005723,0.000000,0.000000,log_loss,10,"{'criterion': 'log_loss', 'n_estimators': 10}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5,0.283837,0.050500,0.000000,0.000000,log_loss,100,"{'criterion': 'log_loss', 'n_estimators': 100}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
